<a style="float:left;" href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-Detection_Objets/Detection_Objets-colab.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<br/>
### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.
* Pour obtenir de l'information sur une fonction, utilisez la commande Python `help(`"nom de la fonction"`)`

SVP, déployez toutes les cellules en sélectionnant l'item « Développer les rubriques » de l'onglet « Affichage ».

# Détection d'objets

Bien que manquant de temps et d'espace pour traiter sérieusement de la tâche de détection d'objets, voici un petit labo Colab pour vous mettre en appétit sur le sujet plein d'applications.

Rappelons que la détection consiste à identifier les différents objets dans une image en les encadrant dans un rectangle.

La détection d'objets est utilisée pour identifier, isoler et même compter des objets dans une image, cela inclut le comptage du bétail dans un enclos, l'estimation du nombre d'arbres dans une forêt, ou le comptage du nombre de poissons dans une passe migratoire.

## Modèle créé à l'aide de l'IPA de détection d'objets TensorFlow

<p>Utilisation de l'<a href="https://github.com/tensorflow/models/tree/master/research/object_detection" target='_blank'>IPA</a> (<i>API</i>) de détection d'objets TensorFlow. L'IPA de détection d'objets TensorFlow utilise l'algorithme (<i>Faster R-CNN</i>) avec un extracteur d'attributs visuels basé sur l'architecture de réseau convolutif profond <a href="https://paperswithcode.com/method/inception-resnet-v2" target='_blank'>Inception Resnet v2</a>. Le modèle est entraîné par apprentissage par transfert à partir d'un modèle préentraîné sur Imagenet puis peaufiné sur l'ensemble de données <a href="https://cocodataset.org/#home" target='_blank'>COCO</a> 2017 avec des lots de 64 images en couleur de 640 par 640 pixels.</p>

### L'algorithme <code>Faster R-CNN</code>

Voici une tentative d'explication des étapes principales de l'algorithme <code>Faster R-CNN</code>:

<ul>
    <ol>
        <li>Un réseau convolutif profond, d'architecture <a href="https://paperswithcode.com/method/inception-resnet-v2" target='_blank'>Inception Resnet v2</a>, extrait une représentation des attributs visuels de l’image d'entrée;</li> 
        <li>Cette représentation passe à un réseau de proposition de régions (en anglais, <i>region proposal network</i> ou <i>RPN</i>), qui génère des propositions de régions qui sont des rectangles qui délimitent les objets pertinents de l’image. Pour cela, le réseau de proposition de régions utilise des rectangles de taille fixe placés uniformément sur l’image originale afin de détecter les objets;</li>
        <li>Les régions candidates sont ensuite filtrées par un algorithme dit de suppression des non-maxima (en anglais, <i>non-maximum suppression</i>), appelé aussi <a href="https://fr.wikipedia.org/wiki/Filtre_de_Canny" target='_blank'>filtre de Canny</a>, qui ne conserve que les régions les plus intéressantes;</li> 
        <li>La représentation des attributs visuels de l’image d'entrée 1) et les rectangles de délimitation des objets pertinents 3) sont combinées pour générer une nouvelle représentation par sous-échantillonnage (<i>pooling</i>) des régions d'intérêt (en anglais, <i>regions of interest</i> ou <i>RoI</i>);</li>
        <li>Finalement, la représentation des régions d'intérêt passe dans un perceptron multicouche pour la prédiction des coordonnées (par régression) des rectangles contenant des objets et (par classification softmax) les classes d'objets.</li> 
    </ol>
</ul>

### Inspiration et droits d'auteur

Ce laboratoire est une adaptation et modification directe du tutoriel <a href="https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1" target='_blank'>TensorFlow Hub Object Detection</a> - site Google / TensorFlow Hub

##### Copyright (c) 2020, The TensorFlow Hub Authors.
##### Copyright (c) 2022, Claude COULOMBE, modifications et adaptation française

Le contenu de cette page est sous licence <a href="https://creativecommons.org/licenses/by/3.0/deed.fr" target='_blank'>Creative Commons Attribution 3.0 (CC BY 3.0)</a>,<br/>et les exemples de code sont sous <a href="https://www.apache.org/licenses/LICENSE-2.0" target='_blank'>licence Apache 2.0</a>.

#### Données

Les données sur les images du jeux de données <a href="https://cocodataset.org/#home" target='_blank'>COCO</a> (<i>Common Objects in Context</i>) de Microsoft, une banque en données ouvertes sous licence <a href="https://creativecommons.org/licenses/by/4.0/deed.fr" target='_blank'>Creative Commons Attribution 4.0 (CC BY 4.0)</a> de 330 000 photos de 80 objets communs dans leur contexte.</p>

## Fixer le hasard pour la reproductibilité

La mise au point de réseaux de neurones implique certains processus aléatoires. Afin de pouvoir reproduire et comparer vos résultats d'expérience, vous fixez temporairement l'état aléatoire grâce à un germe aléatoire unique.

Pendant la mise au point, vous fixez temporairement l'état aléatoire pour la reproductibilité mais vous répétez l'expérience avec différents germes ou états aléatoires et prenez la moyenne des résultats.
<br/>
**Note** : Pour un système en production, vous ravivez simplement l'état  purement aléatoire avec l'instruction `GERME_ALEATOIRE = None`

In [7]:
# This Colab requires TF 2.5.
# !pip3 install -U "tensorflow>=2.5"

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [8]:
import tensorflow as tf
print("Version TensorFlow:",tf.__version__)

Version TensorFlow: 2.8.0


In [9]:
import os

# Définir un germe aléatoire
GERME_ALEATOIRE = 42

# Définir un état aléatoire pour Python
os.environ['PYTHONHASHSEED'] = str(GERME_ALEATOIRE)

# Définir un état aléatoire pour Python random
import random
random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour NumPy
import numpy as np
np.random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour TensorFlow

tf.random.set_seed(GERME_ALEATOIRE)

# Note: Retrait du comportement déterministe
# à cause de keras.layers.RandomContrast(...)
# dont il n'existe pas de version déterministe
# os.environ['TF_DETERMINISTIC_OPS'] = '1'
# os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print("Version ")
print("Germe aléatoire fixé")

Version 
Germe aléatoire fixé


In [11]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

print("Bibliothèques importées")

Bibliothèques importées


## Utilitaires

Exécutez le code de la cellule suivante pour créer des utilitairesé

* Méthode pour charger une image
* Liste de tous les modèles de TensorFlow Hub
* Liste des tuples avec Human Keypoints pour le jeu de données COCO 2017. Ceci est nécessaire pour les modèles avec des points clés.

In [15]:
# @title Run this!!

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
        path: the file path to the image
    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    image = None
    if(path.startswith('http')):
        response = urlopen(path)
        image_data = response.read()
        image_data = BytesIO(image_data)
        image = Image.open(image_data)
    else:
        image_data = tf.io.gfile.GFile(path, 'rb').read()
        image = Image.open(BytesIO(image_data))

    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((1, im_height, im_width, 3)).astype(np.uint8)


ALL_MODELS = {
    'CenterNet HourGlass104 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512/1',
    'CenterNet HourGlass104 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512_kpts/1',
    'CenterNet HourGlass104 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024/1',
    'CenterNet HourGlass104 Keypoints 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024_kpts/1',
    'CenterNet Resnet50 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512/1',
    'CenterNet Resnet50 V1 FPN Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512_kpts/1',
    'CenterNet Resnet101 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet101v1_fpn_512x512/1',
    'CenterNet Resnet50 V2 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512/1',
    'CenterNet Resnet50 V2 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512_kpts/1',
    'EfficientDet D0 512x512' : 'https://tfhub.dev/tensorflow/efficientdet/d0/1',
    'EfficientDet D1 640x640' : 'https://tfhub.dev/tensorflow/efficientdet/d1/1',
    'EfficientDet D2 768x768' : 'https://tfhub.dev/tensorflow/efficientdet/d2/1',
    'EfficientDet D3 896x896' : 'https://tfhub.dev/tensorflow/efficientdet/d3/1',
    'EfficientDet D4 1024x1024' : 'https://tfhub.dev/tensorflow/efficientdet/d4/1',
    'EfficientDet D5 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d5/1',
    'EfficientDet D6 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d6/1',
    'EfficientDet D7 1536x1536' : 'https://tfhub.dev/tensorflow/efficientdet/d7/1',
    'SSD MobileNet v2 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2',
    'SSD MobileNet V1 FPN 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v1/fpn_640x640/1',
    'SSD MobileNet V2 FPNLite 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1',
    'SSD MobileNet V2 FPNLite 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_640x640/1',
    'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_640x640/1',
    'SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_1024x1024/1',
    'SSD ResNet101 V1 FPN 640x640 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_640x640/1',
    'SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_1024x1024/1',
    'SSD ResNet152 V1 FPN 640x640 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_640x640/1',
    'SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_1024x1024/1',
    'Faster R-CNN ResNet50 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1',
    'Faster R-CNN ResNet50 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_1024x1024/1',
    'Faster R-CNN ResNet50 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_800x1333/1',
    'Faster R-CNN ResNet101 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_640x640/1',
    'Faster R-CNN ResNet101 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_1024x1024/1',
    'Faster R-CNN ResNet101 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_800x1333/1',
    'Faster R-CNN ResNet152 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_640x640/1',
    'Faster R-CNN ResNet152 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_1024x1024/1',
    'Faster R-CNN ResNet152 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_800x1333/1',
    'Faster R-CNN Inception ResNet V2 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1',
    'Faster R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1',
    'Mask R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1'
}

IMAGES_FOR_TEST = {
  'Beach' : 'models/research/object_detection/test_images/image2.jpg',
  'Dogs' : 'models/research/object_detection/test_images/image1.jpg',
  # By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
  'Naxos Taverna' : 'https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg',
  # Source: https://commons.wikimedia.org/wiki/File:The_Coleoptera_of_the_British_islands_(Plate_125)_(8592917784).jpg
  'Beatles' : 'https://upload.wikimedia.org/wikipedia/commons/1/1b/The_Coleoptera_of_the_British_islands_%28Plate_125%29_%288592917784%29.jpg',
  # By Américo Toledano, Source: https://commons.wikimedia.org/wiki/File:Biblioteca_Maim%C3%B3nides,_Campus_Universitario_de_Rabanales_007.jpg
  'Phones' : 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg',
  # Source: https://commons.wikimedia.org/wiki/File:The_smaller_British_birds_(8053836633).jpg
  'Birds' : 'https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg',
}

COCO17_HUMAN_POSE_KEYPOINTS = [  (0, 1),
                                 (0, 2),
                                 (1, 3),
                                 (2, 4),
                                 (0, 5),
                                 (0, 6),
                                 (5, 7),
                                 (7, 9),
                                 (6, 8),
                                 (8, 10),
                                 (5, 6),
                                 (5, 11),
                                 (6, 12),
                                 (11, 12),
                                 (11, 13),
                                 (13, 15),
                                 (12, 14),
                                 (14, 16)
                              ]

print("Utilitaires chargés")


Utilitaires chargés


## Outils de visualisation

Pour visualiser les images avec les rectangles, les points clés et la segmentation détectés appropriés, nous utiliserons l'API de détection d'objets TensorFlow. Pour l'installer, nous allons cloner le dépôt GitHub.

In [16]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3314, done.
remote: Counting objects: 100% (3314/3314), done.
remote: Compressing objects: 100% (2737/2737), done.
remote: Total 3314 (delta 879), reused 1393 (delta 528), pack-reused 0
Receiving objects: 100% (3314/3314), 34.29 MiB | 10.29 MiB/s, done.
Resolving deltas: 100% (879/879), done.
Updating files: 100% (3016/3016), done.


### Intalling the Object Detection API

sudo apt install -y protobuf-compiler

In [22]:
%%bash
apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


apt: invalid flag: install
Usage: apt <apt and javac options> <source files>
where apt options include:
  -classpath <path>          Specify where to find user class files and annotation processor factories
  -cp <path>                 Specify where to find user class files and annotation processor factories
  -d <path>                  Specify where to place processor and javac generated class files
  -s <path>                  Specify where to place processor generated source files
  -source <release>          Provide source compatibility with specified release
  -version                   Version information
  -help                      Print a synopsis of standard options; use javac -help for more options
  -X                         Print a synopsis of nonstandard options
  -J<flag>                   Pass <flag> directly to the runtime system
  -A[key[=value]]            Options to pass to annotation processors
  -nocompile                 Do not compile source files to class file

CalledProcessError: Command 'b'apt install -y protobuf-compiler\ncd models/research/\nprotoc object_detection/protos/*.proto --python_out=.\ncp object_detection/packages/tf2/setup.py .\npython -m pip install .\n'' returned non-zero exit status 1.